In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TerminateOnNaN, TensorBoard
from tensorflow.keras import activations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
import numpy as np

# Load CSV data
train_data = pd.read_csv(r"D:\Projects\Digit-Recognizer\train.csv")

# Prepare CSV dataset
x_csv = train_data.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0
y_csv = to_categorical(train_data['label'].values)

# Load MNIST dataset
(x_mnist_train, y_mnist_train), (x_mnist_test, y_mnist_test) = mnist.load_data()
x_mnist_train = x_mnist_train.reshape(-1, 28, 28, 1) / 255.0
y_mnist_train = to_categorical(y_mnist_train, 10)

x_mnist_test = x_mnist_test.reshape(-1, 28, 28, 1) / 255.0
y_mnist_test = to_categorical(y_mnist_test, 10)

# Concatenate CSV and MNIST datasets
x_combined = np.concatenate([x_csv, x_mnist_train, x_mnist_test], axis=0)
y_combined = np.concatenate([y_csv, y_mnist_train, y_mnist_test], axis=0)

# Train-validation split for combined dataset
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(x_combined, y_combined, test_size=0.1, random_state=random_seed)

# Define the model
model = Sequential([
    ZeroPadding2D(padding=(1, 1), input_shape=(28, 28, 1)),  # Input layer, padding added
    Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Activation(activations.relu),  # Activation layer
    MaxPooling2D(pool_size=(2, 2)),  # Max pooling layer
    ZeroPadding2D(padding=(1, 1)),  # Zero padding layer
    Dropout(0.2),  # Dropout layer
    
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Activation(activations.relu),  # Activation layer
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Max pooling layer
    Dropout(0.2),  # Dropout layer
    
    Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),  # Convolutional layer
    BatchNormalization(),  # Batch normalization layer
    Activation(activations.relu),  # Activation layer
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Max pooling layer
    Dropout(0.3),  # Dropout layer
    
    Flatten(),  # Flatten the 3D outputs to 1D
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.4),  # Dropout layer
    Dense(10, activation='softmax')  # Output layer for classification
])

# Callbacks for saving the best model and learning rate reduction
checkpoint = ModelCheckpoint(
    r'best_model.keras', 
    monitor='val_accuracy',
    save_best_only=True, 
    verbose=1,
    mode='min'
)
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy', 
    patience=3, 
    verbose=1, 
    factor=0.5, 
    min_lr=0.00001
)
early_stopping = EarlyStopping(
    monitor='val_loss',        # Моніторинг валідаційних втрат
    patience=5,                # Чекати 5 епох без покращення перед зупинкою
    restore_best_weights=True,  # Відновити ваги найкращої моделі
    mode='auto'
)
terminate_nan = TerminateOnNaN()
tensorboard = TensorBoard(
    log_dir='./public',         # Директорія для збереження логів
    histogram_freq=1,        # Записувати гістограми змінних після кожної епохи
    write_graph=True,        # Записувати граф моделі
    write_images=True,      # Записувати зображення ваг
    update_freq='epoch'     # Оновлення на основі кожної епохи
)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Image Data Generator for augmentation
datagen = ImageDataGenerator(
   rotation_range=10,          # Randomly rotate images by up to 10 degrees
   zoom_range=0.1,             # Randomly zoom images by up to 10%
   width_shift_range=0.1,      # Randomly shift images horizontally by up to 10%
   height_shift_range=0.1,     # Randomly shift images vertically by up to 10%
)

# Train the model
batch_size = 100
history = model.fit(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    epochs=10,
                    validation_data=(X_val, Y_val),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    verbose=1,
                    callbacks=[checkpoint, learning_rate_reduction, early_stopping, terminate_nan, tensorboard],
                    shuffle=True)

# Evaluate the model
model.evaluate(X_val, Y_val)

# Plot accuracy and loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [3]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d_2                │ (None, 30, 30, 1)      │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 30, 30, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 30, 30, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_3                │ (None, 17, 17, 32)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 17, 17, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 17, 17, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 17, 17, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 17, 17, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 17, 17, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 8, 8, 256)      │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 8, 8, 256)      │             

 Total params: 4,041,792 (15.42 MB)

 Trainable params: 1,346,794 (5.14 MB)

 Non-trainable params: 1,408 (5.50 KB)

 Optimizer params: 2,693,590 (10.28 MB)